# Neural and Pre-Trained Machine Translation

- 📺 **Video:** [https://youtu.be/bcP4b_4HQ8A](https://youtu.be/bcP4b_4HQ8A)

## Overview
- Review encoder-decoder architectures with attention and modern pre-trained MT models.
- Discuss transfer from multilingual pre-training (mBART, mT5).

## Key ideas
- **Seq2seq with attention:** decoder attends to encoder states for every target token.
- **Pre-training:** multilingual denoising yields strong initialization.
- **Fine-tuning:** adapt pre-trained models to domain-specific parallel data.
- **Evaluation:** monitor BLEU and human adequacy for domain shifts.

## Demo
Train a miniature attention-based seq2seq model on synthetic pairs to illustrate attention weights, following the lecture (https://youtu.be/jjaJrCB3Q6c).

In [1]:
import torch
from torch import nn

pairs = [
    ('bonjour', 'hello'),
    ('salut', 'hi'),
    ('merci', 'thanks')
]

chars = sorted(set(''.join(src for src, _ in pairs) + ''.join(tgt for _, tgt in pairs))) + ['<pad>', '<s>', '</s>']
char_to_id = {c: i for i, c in enumerate(chars)}
all_texts = [src for src, _ in pairs] + [tgt for _, tgt in pairs]
max_len = max(len(text) for text in all_texts) + 2

def encode(text):
    ids = [char_to_id['<s>']] + [char_to_id[c] for c in text] + [char_to_id['</s>']]
    ids += [char_to_id['<pad>']] * (max_len - len(ids))
    return ids

src = torch.tensor([encode(s) for s, _ in pairs])
tgt = torch.tensor([encode(t) for _, t in pairs])

embed = nn.Embedding(len(chars), 32)
encoder = nn.GRU(32, 32, batch_first=True)
decoder = nn.GRU(32, 32, batch_first=True)
attn = nn.Linear(32 * 2, 1)
out = nn.Linear(32, len(chars))
criterion = nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
optimizer = torch.optim.Adam(list(embed.parameters()) + list(encoder.parameters()) + list(decoder.parameters()) + list(attn.parameters()) + list(out.parameters()), lr=5e-3)

for epoch in range(1, 201):
    enc_emb = embed(src)
    enc_outputs, hidden = encoder(enc_emb)
    dec_emb = embed(tgt[:, :-1])
    dec_outputs, _ = decoder(dec_emb, hidden)
    contexts = []
    for t_step in range(dec_outputs.size(1)):
        query = dec_outputs[:, t_step:t_step+1, :].expand_as(enc_outputs)
        score = attn(torch.cat([enc_outputs, query], dim=-1))
        weights = torch.softmax(score, dim=1)
        context = (weights * enc_outputs).sum(dim=1)
        contexts.append(context)
    contexts = torch.stack(contexts, dim=1)
    logits = out(contexts)
    loss = criterion(logits.reshape(-1, len(chars)), tgt[:, 1:].reshape(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print(f"epoch {epoch:3d} | loss {loss.item():.4f}")


epoch  50 | loss 1.6629
epoch 100 | loss 1.6478


epoch 150 | loss 1.6455
epoch 200 | loss 1.6445


## Try it
- Modify the demo
- Add a tiny dataset or counter-example


## References
- [Eisenstein 18.1](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Eisenstein 18.1](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [HMM-Based Word Alignment in Statistical Translation](https://www.aclweb.org/anthology/C96-2141.pdf)
- [Pharaoh: A Beam Search Decoder for Phrase-Based Statistical Machine Translation Models](http://homepages.inf.ed.ac.uk/pkoehn/publications/pharaoh-amta2004.pdf)
- [Minimum Error Rate Training in Statistical Machine Translation](https://www.aclweb.org/anthology/P03-1021/)
- [Eisenstein 18.4](https://github.com/jacobeisenstein/gt-nlp-class/blob/master/notes/eisenstein-nlp-notes.pdf)
- [Revisiting Low-Resource Neural Machine Translation: A Case Study](https://arxiv.org/abs/1905.11901)
- [In Neural Machine Translation, What Does Transfer Learning Transfer?](https://aclanthology.org/2020.acl-main.688/)
- [Multilingual Denoising Pre-training for Neural Machine Translation](https://arxiv.org/abs/2001.08210)
- [Large Language Models Are State-of-the-Art Evaluators of Translation Quality](https://arxiv.org/abs/2302.14520)
- [The use of MMR, diversity-based reranking for reordering documents and producing summaries](https://dl.acm.org/doi/10.1145/290941.291025)
- [LexRank: Graph-based Lexical Centrality as Salience in Text Summarization](https://arxiv.org/abs/1109.2128)
- [A Scalable Global Model for Summarization](https://www.aclweb.org/anthology/W09-1802/)
- [Revisiting the Centroid-based Method: A Strong Baseline for Multi-Document Summarization](https://www.aclweb.org/anthology/W17-4511/)
- [BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension](https://www.aclweb.org/anthology/2020.acl-main.703/)
- [PEGASUS: Pre-training with Extracted Gap-sentences for Abstractive Summarization](https://arxiv.org/abs/1912.08777)
- [Evaluating Factuality in Generation with Dependency-level Entailment](https://arxiv.org/pdf/2010.05478.pdf)
- [Asking and Answering Questions to Evaluate the Factual Consistency of Summaries](https://arxiv.org/abs/2004.04228)
- [News Summarization and Evaluation in the Era of GPT-3](https://arxiv.org/abs/2209.12356)
- [Unsupervised Part-of-Speech Tagging with Bilingual Graph-Based Projections](https://www.aclweb.org/anthology/P11-1061/)
- [Multi-Source Transfer of Delexicalized Dependency Parsers](https://www.aclweb.org/anthology/D11-1006/)
- [Massively Multilingual Word Embeddings](https://arxiv.org/pdf/1602.01925.pdf)
- [Massively Multilingual Sentence Embeddings for Zero-Shot Cross-Lingual Transfer and Beyond](https://www.aclweb.org/anthology/Q19-1038.pdf)
- [How multilingual is Multilingual BERT?](https://www.aclweb.org/anthology/P19-1493.pdf)
- [Climbing towards NLU: On Meaning, Form, and Understanding in the Age of Data](https://aclanthology.org/2020.acl-main.463/)
- [Provable Limitations of Acquiring Meaning from Ungrounded Form: What Will Future Language Models Understand?](https://arxiv.org/abs/2104.10809)
- [Entailment Semantics Can Be Extracted from an Ideal Language Model](https://arxiv.org/abs/2209.12407)
- [Experience Grounds Language](https://arxiv.org/abs/2004.10151)
- [VQA: Visual Question Answering](https://arxiv.org/abs/1505.00468)
- [Learning Transferable Visual Models From Natural Language Supervision](https://arxiv.org/abs/2103.00020)
- [The Social Impact of Natural Language Processing](https://aclanthology.org/P16-2096.pdf)
- [Men Also Like Shopping: Reducing Gender Bias Amplification using Corpus-level Constraints](https://arxiv.org/pdf/1707.09457.pdf)
- [GeoMLAMA: Geo-Diverse Commonsense Probing on Multilingual Pre-Trained Language Models](https://arxiv.org/abs/2205.12247)
- [Visually Grounded Reasoning across Languages and Cultures](https://arxiv.org/abs/2109.13238)
- [On the Dangers of Stochastic Parrots: Can Language Models Be Too Big?](https://dl.acm.org/doi/10.1145/3442188.3445922)
- [RealToxicityPrompts: Evaluating Neural Toxic Degeneration in Language Models](https://arxiv.org/abs/2009.11462)
- [Datasheets for Datasets](https://arxiv.org/pdf/1803.09010.pdf)
- [Closing the AI Accountability Gap: Defining an End-to-End Framework for Internal Algorithmic Auditing](https://dl.acm.org/doi/pdf/10.1145/3351095.3372873)


*Links only; we do not redistribute slides or papers.*